# Neo4j 

## Cours 101

Concepts de base


In [1]:
from py2neo import Graph

graph = Graph()
graph.delete_all()

In [2]:
query = '''
LOAD CSV WITH HEADERS FROM "https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr_referentiel_metier_referens_3/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true" AS line FIELDTERMINATOR ';'
MERGE 
(bap:BAP 
	{code: line.`Code de la branche d’activité professionnelle`,
	 label: line.`Branche d’activité professionnelle`}
)
MERGE (codeEmploiType:EmploiType {
	code: line.`Code emploi type`,
    label: line.`Intitulé de l’emploi type`
})
MERGE (codeEmploiType)-[:IS_IN]->(bap)
    
return bap, codeEmploiType'''
graph.run(query)

In [6]:
import sys
sys.path

['',
 '/home/mdexet/Workdir/loops_neo4j_presentation/lib/python36.zip',
 '/home/mdexet/Workdir/loops_neo4j_presentation/lib/python3.6',
 '/home/mdexet/Workdir/loops_neo4j_presentation/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/mdexet/Workdir/loops_neo4j_presentation/lib/python3.6/site-packages',
 '/home/mdexet/Workdir/loops_neo4j_presentation/lib/python3.6/site-packages/IPython/extensions',
 '/home/mdexet/.ipython']